<a href="https://colab.research.google.com/github/hamedebrahimi68/samtat_reporter/blob/master/bitcoin_2h_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from datetime import datetime, timedelta

In [3]:
# دریافت داده‌های بیت‌کوین از یاهو فایننس
def get_bitcoin_data():
    btc = yf.Ticker("BTC-USD")
    data = btc.history(period="2y", interval="1h")
    return data['Close']

In [4]:
# آماده‌سازی داده‌ها برای مدل LSTM
def prepare_data(data, time_steps=12):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

    X, y = [], []
    for i in range(len(scaled_data) - time_steps):
        X.append(scaled_data[i:i+time_steps])
        y.append(scaled_data[i+time_steps])

    return np.array(X), np.array(y), scaler

In [5]:
# ساخت مدل LSTM
def create_model(time_steps):
    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=(time_steps, 1)),
        Dropout(0.2),
        LSTM(units=50),
        Dropout(0.2),
        Dense(1)])
    model.compile(optimizer='adam', loss='mse')
    return model


In [6]:
# پیش‌بینی قیمت
def predict_price(model, data, scaler, time_steps=12):
    last_sequence = data[-time_steps:].values.reshape(-1, 1)
    last_sequence = scaler.transform(last_sequence)
    last_sequence = last_sequence.reshape(1, time_steps, 1)

    predicted_scaled = model.predict(last_sequence)
    predicted_price = scaler.inverse_transform(predicted_scaled)[0][0]

    return predicted_price

In [9]:
def main():
    # دریافت داده‌ها
    print("دریافت داده‌های بیت‌کوین...")
    bitcoin_data = get_bitcoin_data()

    # آماده‌سازی داده‌ها
    TIME_STEPS = 12  # 12 ساعت قبل برای پیش‌بینی 2 ساعت بعد
    X, y, scaler = prepare_data(bitcoin_data, TIME_STEPS)

    # تقسیم داده‌ها به train و test
    train_size = int(len(X)*0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # ساخت و آموزش مدل
    print("آموزش مدل LSTM...")
    model = create_model(TIME_STEPS)
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

    # پیش‌بینی قیمت برای 2 ساعت آینده
    current_price = bitcoin_data[-1]
    predicted_price = predict_price(model, bitcoin_data, scaler, TIME_STEPS)

    print(f"\nقیمت فعلی بیت‌کوین: ${current_price:.2f}")
    print(f"قیمت پیش‌بینی شده برای 2 ساعت آینده: ${predicted_price:.2f}")
    print(f"درصد تغییر پیش‌بینی شده: {((predicted_price - current_price)/current_price):.2f}%")

In [8]:
main()

دریافت داده‌های بیت‌کوین...
آموزش مدل LSTM...
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


393/393 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 0.0083 - val_loss: 0.0013
Epoch 2/50
393/393 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - loss: 9.2159e-04 - val_loss: 4.1306e-04
Epoch 3/50
393/393 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 0.0010 - val_loss: 1.5037e-04
Epoch 4/50
393/393 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 6.7448e-04 - val_loss: 1.0116e-04
Epoch 5/50
393/393 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 6.5364e-04 - val_loss: 1.0007e-04
Epoch 6/50
393/393 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 5.4142e-04 - val_loss: 1.0483e-04
Epoch 7/50
393/393 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 4.7515e-04 - val_loss: 7.2886e-04
Epoch 8/50
393/393 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 4.9683e-04 - val_loss: 1.8465e-04
Epoch 9/50
393/393 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 4.4587e-04 - val_loss: 2.3141e-04
Epoch 10/50
393/393 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 3.9413e-04 - val_loss: 6.2458e-04
Epoch 11/50
393/393 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 3.9822e-

<ipython-input-7-9c33aa454ee4>:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = bitcoin_data[-1]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step

قیمت فعلی بیت‌کوین: $76383.42
قیمت پیش‌بینی شده برای 2 ساعت آینده: $74065.69
درصد تغییر پیش‌بینی شده: -0.03%
